In [23]:
#Import all the necessary libraries

import pandas as pd  
import os                
from datetime import datetime
import time
import matplotlib.pyplot as plt
from math import sin, cos, tan
import matplotlib.dates as mdates
import pgeocode

In [24]:
#Load in all of the data files and combine them into a single DataFrame
# Directory where your files are located
directory = './Data/'
dfs = []

#loop through all the files in a data directory
for filename in os.listdir(directory):
    #ignore a file start with the name weekly, thats the mortageg rates data 
    if not filename.startswith("weekly"):
        #read file path 
        filepath = os.path.join(directory, filename)
        #read a file in a dataframe
        df = pd.read_csv(filepath, encoding='ISO-8859-1')
        #append the file in dictionary 
        dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)    
combined_df.head(10)

,MLS #,Class,Property Type,Address,City,Zip,Neighborhood,Subdivision,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,Days On Market,List Price,Sold Price
0,LP649830,RESIDENTIAL,Single Family Residence,2263 Shaw Rd Extension,Fayetteville,28311,NaN,NaN,3.0,1,0,NaN,1977.0,2/9/2021,3/11/2021,9,"$79,900",$79
1,2354950,RESIDENTIAL,Single Family Residence,97 Allie Clay Road,Roxboro,27573,NaN,Not in a Subdivision,3.0,1,"1,118",.51-.75 Acres,1934.0,11/22/2020,1/12/2021,13,"$40,000","$15,000"
2,LP630668,RESIDENTIAL,Manufactured,146 Ralph Hunt Boulevard,Orrum,28369,NaN,NaN,3.0,2,0,NaN,1995.0,4/21/2020,2/1/2021,254,"$36,000","$17,000"
3,LP630661,RESIDENTIAL,Manufactured,163 Ralph Hunt Boulevard,Orrum,28369,NaN,NaN,3.0,2,0,NaN,1998.0,4/21/2020,2/1/2021,254,"$29,000","$17,000"
4,LP624150,RESIDENTIAL,Single Family Residence,405 Mcleod Road,Red Springs,28377,NaN,NaN,2.0,1,0,NaN,1940.0,1/7/2020,1/25/2021,357,"$22,000","$18,500"
5,2365371,RESIDENTIAL,Manufactured,921 West Street,Smithfield,27577,NaN,Not in a Subdivision,3.0,1,650,0-.25 Acres,1972.0,2/5/2021,3/26/2021,1,"$20,000","$20,000"
6,LP630389,RESIDENTIAL,Manufactured,555 Ralph Hunt Boulevard,Orrum,28369,NaN,NaN,3.0,2,0,NaN,1998.0,4/14/2020,2/1/2021,261,"$38,500","$20,000"
7,LP630671,RESIDENTIAL,Manufactured,82 Ralph Hunt Boulevard,Orrum,28369,NaN,NaN,3.0,2,0,NaN,1997.0,4/21/2020,2/1/2021,254,"$39,000","$20,000"
8,LP648914,RESIDENTIAL,Single Family Residence,216 Register Street,Sanford,27330,NaN,Hillcrest,0.0,0,0,NaN,1930.0,1/21/2021,2/10/2021,12,"$34,000","$20,000"
9,116488,RESIDENTIAL,Single Family Residence,413 NE Market Street,Reidsville,27320,NaN,NaN,2.0,2,0,.51-.75 Acres,1900.0,2/11/2021,3/26/2021,25,"$31,500","$22,000"


In [25]:
#Checking how many values are null per column
combined_df.isnull().sum()

MLS #                          0
Class                          0
Property Type                  0
Address                        0
City                           0
Zip                           42
Neighborhood              162151
Subdivision                17877
Bedrooms                      18
Total Baths                    0
Total Living Area SqFt         0
Acres                      10856
Year Built                    17
List Date                      0
Closing Date                   0
Days On Market                 0
List Price                     0
Sold Price                     0
dtype: int64

In [26]:
#Checking how many values are null per column
combined_df.describe

<bound method NDFrame.describe of            MLS #        Class            Property Type  \
0       LP649830  RESIDENTIAL  Single Family Residence   
1        2354950  RESIDENTIAL  Single Family Residence   
2       LP630668  RESIDENTIAL             Manufactured   
3       LP630661  RESIDENTIAL             Manufactured   
4       LP624150  RESIDENTIAL  Single Family Residence   
...          ...          ...                      ...   
168792   2473018  RESIDENTIAL  Single Family Residence   
168793  10003230  RESIDENTIAL  Single Family Residence   
168794   2490937  RESIDENTIAL  Single Family Residence   
168795  10029216  RESIDENTIAL  Single Family Residence   
168796  10009864  RESIDENTIAL  Single Family Residence   

                         Address          City    Zip Neighborhood  \
0         2263 Shaw Rd Extension  Fayetteville  28311          NaN   
1             97 Allie Clay Road       Roxboro  27573          NaN   
2       146 Ralph Hunt Boulevard         Orrum  28369      

In [27]:
#Checking df Info
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168797 entries, 0 to 168796
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   MLS #                   168797 non-null  object 
 1   Class                   168797 non-null  object 
 2   Property Type           168797 non-null  object 
 3   Address                 168797 non-null  object 
 4   City                    168797 non-null  object 
 5   Zip                     168755 non-null  object 
 6   Neighborhood            6646 non-null    object 
 7   Subdivision             150920 non-null  object 
 8   Bedrooms                168779 non-null  float64
 9   Total Baths             168797 non-null  int64  
 10  Total Living Area SqFt  168797 non-null  object 
 11  Acres                   157941 non-null  object 
 12  Year Built              168780 non-null  float64
 13  List Date               168797 non-null  object 
 14  Closing Date        

In [28]:
combined_df.columns

Index(['MLS #', 'Class', 'Property Type', 'Address', 'City', 'Zip',
       'Neighborhood', 'Subdivision', 'Bedrooms', 'Total Baths',
       'Total Living Area SqFt', 'Acres', 'Year Built', 'List Date',
       'Closing Date', 'Days On Market', 'List Price', 'Sold Price'],
      dtype='object')

In [31]:
# Drop the 'Neighborhood' column
combined_df = combined_df.drop('Neighborhood', axis=1)

# Drop the 'Subdivision' column
combined_df = combined_df.drop('Subdivision', axis=1)

combined_df.isnull().sum()

MLS #                         0
Class                         0
Property Type                 0
Address                       0
City                          0
Zip                          42
Bedrooms                     18
Total Baths                   0
Total Living Area SqFt        0
Acres                     10856
Year Built                   17
List Date                     0
Closing Date                  0
Days On Market                0
List Price                    0
Sold Price                    0
dtype: int64

In [32]:
#Drop the rows with missing values
combined_df.dropna(inplace=True)

combined_df

,MLS #,Class,Property Type,Address,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,Days On Market,List Price,Sold Price
1,2354950,RESIDENTIAL,Single Family Residence,97 Allie Clay Road,Roxboro,27573,3.0,1,"1,118",.51-.75 Acres,1934.0,11/22/2020,1/12/2021,13,"$40,000","$15,000"
5,2365371,RESIDENTIAL,Manufactured,921 West Street,Smithfield,27577,3.0,1,650,0-.25 Acres,1972.0,2/5/2021,3/26/2021,1,"$20,000","$20,000"
9,116488,RESIDENTIAL,Single Family Residence,413 NE Market Street,Reidsville,27320,2.0,2,0,.51-.75 Acres,1900.0,2/11/2021,3/26/2021,25,"$31,500","$22,000"
11,2355534,RESIDENTIAL,Single Family Residence,710 Water Street,Henderson,27536,2.0,1,"1,012",0-.25 Acres,1984.0,11/29/2020,1/15/2021,17,"$40,000","$24,000"
12,2360306,RESIDENTIAL,Single Family Residence,1918 Powell Street,Fayetteville,28306,2.0,1,859,0-.25 Acres,1940.0,1/6/2021,1/22/2021,5,"$25,000","$24,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168792,2473018,RESIDENTIAL,Single Family Residence,250 Blue Violet Way,Durham,27713,5.0,9,"10,850",6-10.9 Acres,2008.0,9/8/2022,5/1/2024,587,"$4,995,000","$4,415,000"
168793,10003230,RESIDENTIAL,Single Family Residence,1711 Hunting Ridge Road,Raleigh,27615,4.0,6,"5,951",.51-.75 Acres,2023.0,12/22/2023,4/11/2024,0,"$5,500,000","$5,000,000"
168794,2490937,RESIDENTIAL,Single Family Residence,4821 Parker Meadow Drive,Raleigh,27614,6.0,10,"8,074",1-2.9 Acres,2023.0,1/18/2023,5/3/2024,0,"$5,121,787","$5,121,787"
168795,10029216,RESIDENTIAL,Single Family Residence,702 Old School Road,Chapel Hill,27516,5.0,8,"6,852",11+ Acres,2016.0,5/14/2024,5/14/2024,0,"$5,750,000","$5,750,000"


In [33]:
# Replace float Acres values with simplified float values in the 'Acres' column
combined_df['Acres'] = combined_df['Acres'].replace('0-.25 Acres', 0.0)
combined_df['Acres'] = combined_df['Acres'].replace('.26-.5 Acres', 0.26)
combined_df['Acres'] = combined_df['Acres'].replace('.51-.75 Acres', 0.51)
combined_df['Acres'] = combined_df['Acres'].replace('.76-.99 Acres', 0.76)
combined_df['Acres'] = combined_df['Acres'].replace('1-2.9 Acres', 1.0)
combined_df['Acres'] = combined_df['Acres'].replace('3-5.9 Acres', 3.0)
combined_df['Acres'] = combined_df['Acres'].replace('6-10.9 Acres', 6.0)
combined_df['Acres'] = combined_df['Acres'].replace('11+ Acres', 11.0)

#Convert the Acres column to an integer
combined_df['Acres'] = combined_df['Acres'].astype(int)

combined_df['Acres'].unique()


array([ 0,  1,  3,  6, 11])

In [34]:
#Convert the Year Build column to an integer
combined_df['Year Built'] = combined_df['Year Built'].astype(int)

In [35]:
#Covert Zip to a string so we can slice it
combined_df['Zip'] = combined_df['Zip'].astype(str)

# Slice the 'Zip' column to just the first 5 characters
combined_df['Zip'] = combined_df['Zip'].str.slice(0, 5)

#Convert the Zip column to an integer
combined_df['Zip'] = combined_df['Zip'].astype(int)

#combined_df.isnull().sum()
#combined_df['Zip'].unique()



In [36]:
# Convert the 'List Date' and 'Closing Date' columns to datetime
combined_df['List Date'] = pd.to_datetime(combined_df['List Date'])
combined_df['Closing Date'] = pd.to_datetime(combined_df['Closing Date'])

# Calculate the number of days between 'List Date' and 'Closing Date'
combined_df['Days Between'] = (combined_df['Closing Date'] - combined_df['List Date']).dt.days

combined_df

,MLS #,Class,Property Type,Address,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,Days On Market,List Price,Sold Price,Days Between
1,2354950,RESIDENTIAL,Single Family Residence,97 Allie Clay Road,Roxboro,27573,3.0,1,"1,118",0,1934,2020-11-22,2021-01-12,13,"$40,000","$15,000",51
5,2365371,RESIDENTIAL,Manufactured,921 West Street,Smithfield,27577,3.0,1,650,0,1972,2021-02-05,2021-03-26,1,"$20,000","$20,000",49
9,116488,RESIDENTIAL,Single Family Residence,413 NE Market Street,Reidsville,27320,2.0,2,0,0,1900,2021-02-11,2021-03-26,25,"$31,500","$22,000",43
11,2355534,RESIDENTIAL,Single Family Residence,710 Water Street,Henderson,27536,2.0,1,"1,012",0,1984,2020-11-29,2021-01-15,17,"$40,000","$24,000",47
12,2360306,RESIDENTIAL,Single Family Residence,1918 Powell Street,Fayetteville,28306,2.0,1,859,0,1940,2021-01-06,2021-01-22,5,"$25,000","$24,000",16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168792,2473018,RESIDENTIAL,Single Family Residence,250 Blue Violet Way,Durham,27713,5.0,9,"10,850",6,2008,2022-09-08,2024-05-01,587,"$4,995,000","$4,415,000",601
168793,10003230,RESIDENTIAL,Single Family Residence,1711 Hunting Ridge Road,Raleigh,27615,4.0,6,"5,951",0,2023,2023-12-22,2024-04-11,0,"$5,500,000","$5,000,000",111
168794,2490937,RESIDENTIAL,Single Family Residence,4821 Parker Meadow Drive,Raleigh,27614,6.0,10,"8,074",1,2023,2023-01-18,2024-05-03,0,"$5,121,787","$5,121,787",471
168795,10029216,RESIDENTIAL,Single Family Residence,702 Old School Road,Chapel Hill,27516,5.0,8,"6,852",11,2016,2024-05-14,2024-05-14,0,"$5,750,000","$5,750,000",0


In [37]:
# Count the number of 0 values in the 'Total Living Area SqFt' column
num_zeros = combined_df['Total Living Area SqFt'].value_counts().get(0, 0)
print(num_zeros)

16387


C:\Users\David\AppData\Local\Temp\ipykernel_13696\4168313785.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_zeros = combined_df['Total Living Area SqFt'].value_counts().get(0, 0)


In [38]:
#Convert the Bedrooms column to an integer
combined_df['Bedrooms'] = combined_df['Bedrooms'].astype(int)
combined_df['Bedrooms'].unique()

array([ 3,  2,  4,  1,  5, 10,  6,  8,  7,  0, 11, 13, 14, 43,  9, 44])

In [39]:
#Convert the Total Baths column to an integer
combined_df['Total Baths'] = combined_df['Total Baths'].astype(int)
combined_df['Total Baths'].unique()

array([ 1,  2,  3,  4,  5,  7,  6,  0,  8,  9, 10, 12, 17, 32, 14, 23, 26,
       11, 34, 22])

In [40]:
#Define the number of zeros in the 'Total Baths' column
num_zeros = combined_df['Total Baths'].value_counts().get(0, 0)
print(num_zeros)

50


In [41]:
# Drop all the rows that have a 0 in the 'Total Baths' column
combined_df = combined_df[combined_df['Total Baths'] != 0]

#Recheck the number of zeros in the 'Total Baths' column
num_zeros = combined_df['Total Baths'].value_counts().get(0, 0)
print(num_zeros)

0


In [42]:
#Covert Sqft to a string so we can remove the commas
combined_df['Total Living Area SqFt'] = combined_df['Total Living Area SqFt'].astype(str)

# Remove commas from the 'Total Living Area SqFt' column
combined_df['Total Living Area SqFt'] = combined_df['Total Living Area SqFt'].str.replace(',', '')

# Convert the 'Total Living Area SqFt' column to an integer
combined_df['Total Living Area SqFt'] = combined_df['Total Living Area SqFt'].astype(int)

# Drop all the rows that have a 0 in the 'Total Baths' column
combined_df = combined_df[combined_df['Total Living Area SqFt'] != 0]

num_zeros = combined_df['Total Living Area SqFt'].value_counts().get(0, 0)
print(num_zeros)


0


C:\Users\David\AppData\Local\Temp\ipykernel_13696\4065630321.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Total Living Area SqFt'] = combined_df['Total Living Area SqFt'].astype(str)
C:\Users\David\AppData\Local\Temp\ipykernel_13696\4065630321.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['Total Living Area SqFt'] = combined_df['Total Living Area SqFt'].str.replace(',', '')
C:\Users\David\AppData\Local\Temp\ipykernel_13696\4065630321.py:8: SettingWithCopyWarning: 
A val

In [43]:
#Covert List Price and Sold Price to a string so we can remove the dollar sign and commas
combined_df['List Price'] = combined_df['List Price'].astype(str)
combined_df['Sold Price'] = combined_df['Sold Price'].astype(str)

# Remove dollar signs and commas from the 'List Price' and 'Sold Price' column
combined_df['List Price'] = combined_df['List Price'].replace('[$,]', '', regex=True)
combined_df['Sold Price'] = combined_df['Sold Price'].replace('[$,]', '', regex=True)

# Convert the 'List Price' and 'Sold Price' columns to integers
combined_df['List Price'] = combined_df['List Price'].astype(int)
combined_df['Sold Price'] = combined_df['Sold Price'].astype(int)

# Calculate the profit made on each house
combined_df['Profit'] = combined_df['Sold Price'] - combined_df['List Price']

combined_df.head(10)




,MLS #,Class,Property Type,Address,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,Days On Market,List Price,Sold Price,Days Between,Profit
1,2354950,RESIDENTIAL,Single Family Residence,97 Allie Clay Road,Roxboro,27573,3,1,1118,0,1934,2020-11-22,2021-01-12,13,40000,15000,51,-25000
5,2365371,RESIDENTIAL,Manufactured,921 West Street,Smithfield,27577,3,1,650,0,1972,2021-02-05,2021-03-26,1,20000,20000,49,0
11,2355534,RESIDENTIAL,Single Family Residence,710 Water Street,Henderson,27536,2,1,1012,0,1984,2020-11-29,2021-01-15,17,40000,24000,47,-16000
12,2360306,RESIDENTIAL,Single Family Residence,1918 Powell Street,Fayetteville,28306,2,1,859,0,1940,2021-01-06,2021-01-22,5,25000,24000,16,-1000
14,2301316,RESIDENTIAL,Single Family Residence,505 Ivey Street,Henderson,27536,3,2,1100,0,1958,2020-02-11,2021-02-05,309,39000,25000,360,-14000
15,2300217,RESIDENTIAL,Single Family Residence,877 Lamb Street,Henderson,27536,3,1,1086,0,1950,2020-02-05,2021-02-05,315,35000,25000,366,-10000
16,2357601,RESIDENTIAL,Single Family Residence,310 Hall Avenue,Burlington,27217,3,1,1414,0,1919,2020-12-12,2021-02-25,8,29900,25000,75,-4900
17,2369657,RESIDENTIAL,Single Family Residence,504 SE New Bern Street,Wilson,27893,3,1,1395,0,1945,2021-03-02,2021-03-16,2,29900,25000,14,-4900
18,2355248,RESIDENTIAL,Single Family Residence,722 Eaton Street,Henderson,27536,4,3,1675,0,1955,2020-11-24,2021-01-06,30,38000,26000,43,-12000
19,2365135,RESIDENTIAL,Single Family Residence,324 Arch Street,Henderson,27536,3,1,1192,0,1952,2021-02-04,2021-02-25,12,29900,26000,21,-3900


In [44]:
# Count the number of negative numbers in the 'Profit' column
num_negative = combined_df[combined_df['Profit'] < 0].shape[0]
print(num_negative)

42272


In [45]:
# Count the number of numbers in the 'Profit' column that are less than -10000
num_negative = combined_df[combined_df['Profit'] < -100000].shape[0]
num_negative

672

In [46]:
combined_df.head(10)

,MLS #,Class,Property Type,Address,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,Days On Market,List Price,Sold Price,Days Between,Profit
1,2354950,RESIDENTIAL,Single Family Residence,97 Allie Clay Road,Roxboro,27573,3,1,1118,0,1934,2020-11-22,2021-01-12,13,40000,15000,51,-25000
5,2365371,RESIDENTIAL,Manufactured,921 West Street,Smithfield,27577,3,1,650,0,1972,2021-02-05,2021-03-26,1,20000,20000,49,0
11,2355534,RESIDENTIAL,Single Family Residence,710 Water Street,Henderson,27536,2,1,1012,0,1984,2020-11-29,2021-01-15,17,40000,24000,47,-16000
12,2360306,RESIDENTIAL,Single Family Residence,1918 Powell Street,Fayetteville,28306,2,1,859,0,1940,2021-01-06,2021-01-22,5,25000,24000,16,-1000
14,2301316,RESIDENTIAL,Single Family Residence,505 Ivey Street,Henderson,27536,3,2,1100,0,1958,2020-02-11,2021-02-05,309,39000,25000,360,-14000
15,2300217,RESIDENTIAL,Single Family Residence,877 Lamb Street,Henderson,27536,3,1,1086,0,1950,2020-02-05,2021-02-05,315,35000,25000,366,-10000
16,2357601,RESIDENTIAL,Single Family Residence,310 Hall Avenue,Burlington,27217,3,1,1414,0,1919,2020-12-12,2021-02-25,8,29900,25000,75,-4900
17,2369657,RESIDENTIAL,Single Family Residence,504 SE New Bern Street,Wilson,27893,3,1,1395,0,1945,2021-03-02,2021-03-16,2,29900,25000,14,-4900
18,2355248,RESIDENTIAL,Single Family Residence,722 Eaton Street,Henderson,27536,4,3,1675,0,1955,2020-11-24,2021-01-06,30,38000,26000,43,-12000
19,2365135,RESIDENTIAL,Single Family Residence,324 Arch Street,Henderson,27536,3,1,1192,0,1952,2021-02-04,2021-02-25,12,29900,26000,21,-3900


In [47]:
# Sort the DataFrame by the 'Profit' column in descending order
combined_df=combined_df.sort_values('Profit', ascending=False)
combined_df.head(10)    

,MLS #,Class,Property Type,Address,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,Days On Market,List Price,Sold Price,Days Between,Profit
168796,10009864,RESIDENTIAL,Single Family Residence,100 Wall Creek Drive,Rolesville,27571,5,3,2981,0,1999,2024-02-03,2024-06-14,102,669000,6395000,132,5726000
168784,2531856,RESIDENTIAL,Single Family Residence,12 Stonecliff Lane,Sanford,27332,4,3,2017,1,2023,2023-09-12,2024-05-10,0,319999,3249999,241,2930000
168786,2485092,RESIDENTIAL,Single Family Residence,20 E Saint Andrews Drive,Zebulon,27597,3,3,2072,0,2023,2022-11-26,2024-05-15,0,366300,3269300,536,2903000
66276,2372641,RESIDENTIAL,Single Family Residence,2211 Wheeler Road,Raleigh,27607,5,8,9376,0,2022,2021-03-16,2022-02-23,0,1850000,3795000,344,1945000
113526,2410217,RESIDENTIAL,Single Family Residence,7241 Burlington Road,Whitsett,27377,5,7,6983,11,1883,2021-09-27,2023-01-09,102,1300000,3240000,469,1940000
26172,2375877,RESIDENTIAL,Single Family Residence,1129 Shoaf Stone,Wake Forest,27587,5,5,7200,6,2005,2021-04-05,2021-05-27,0,1800000,2900000,52,1100000
93825,2378933,RESIDENTIAL,Single Family Residence,1616 Hunting Ridge Road,Raleigh,27615,5,7,6289,0,2022,2021-04-20,2022-09-28,0,2550000,3577591,526,1027591
148322,2495996,RESIDENTIAL,Single Family Residence,1490 Olives Chapel Road,Apex,27502,4,6,7266,1,2023,2023-02-21,2023-11-21,0,1755900,2598548,273,842648
104029,2293916,RESIDENTIAL,Single Family Residence,106 Lochinvar Court,Cary,27511,5,6,7069,0,2022,2019-12-27,2022-12-30,0,2550000,3296301,1099,746301
80490,2448408,RESIDENTIAL,Single Family Residence,3424 Bellevue Road,Raleigh,27609,5,7,7004,0,1938,2022-05-11,2022-06-01,3,3500000,4200000,21,700000


In [51]:
#Drop the MLS Number, Days On Market, and Class column
#combined_df = combined_df.drop('MLS #', axis=1) - This column is needed for the final output as a UID

#Drop Unnecessary Columns
combined_df = combined_df.drop('Days On Market', axis=1)
combined_df = combined_df.drop('Class', axis=1)
combined_df = combined_df.drop(columns=['Address'])

In [52]:
combined_df.head(10)

,MLS #,Property Type,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,List Price,Sold Price,Days Between,Profit
66276,2372641,Single Family Residence,Raleigh,27607,5,8,9376,0,2022,2021-03-16,2022-02-23,1850000,324999,344,-1525001
113526,2410217,Single Family Residence,Whitsett,27377,5,7,6983,11,1883,2021-09-27,2023-01-09,1300000,3240000,469,1940000
26172,2375877,Single Family Residence,Wake Forest,27587,5,5,7200,6,2005,2021-04-05,2021-05-27,1800000,2900000,52,1100000
93825,2378933,Single Family Residence,Raleigh,27615,5,7,6289,0,2022,2021-04-20,2022-09-28,2550000,3577591,526,1027591
148322,2495996,Single Family Residence,Apex,27502,4,6,7266,1,2023,2023-02-21,2023-11-21,1755900,2598548,273,842648
104029,2293916,Single Family Residence,Cary,27511,5,6,7069,0,2022,2019-12-27,2022-12-30,2550000,3296301,1099,746301
80490,2448408,Single Family Residence,Raleigh,27609,5,7,7004,0,1938,2022-05-11,2022-06-01,3500000,4200000,21,700000
137710,2508624,Single Family Residence,Raleigh,27610,3,2,1450,1,1958,2023-05-03,2023-09-11,249900,850000,131,600100
148321,2428206,Single Family Residence,Apex,27523,4,5,6864,3,2022,2022-01-24,2023-12-18,2000000,2566043,693,566043
40849,2387466,Single Family Residence,Snow Camp,27349,3,4,4689,11,2003,2021-06-03,2021-07-15,1500000,2050000,42,550000


In [53]:

combined_df['Property Type'].unique()

array(['Single Family Residence', 'Condo', 'Manufactured', 'Townhouse',
       'Triplex', 'Duplex', 'Ranch', 'Other', 'Farm', 'Cabin',
       'Quadruplex'], dtype=object)

In [54]:
# Initialize pgeocode Nominatim object for the United States
nomi = pgeocode.Nominatim('us')

# Function to get latitude and longitude
def get_lat_long(zip_code):
    location = nomi.query_postal_code(zip_code)
    return location.latitude, location.longitude

# Apply the function to each zip code
combined_df[['latitude', 'longitude']] = combined_df['Zip'].apply(lambda x: pd.Series(get_lat_long(x)))

combined_df

,MLS #,Property Type,City,Zip,Bedrooms,Total Baths,Total Living Area SqFt,Acres,Year Built,List Date,Closing Date,List Price,Sold Price,Days Between,Profit,latitude,longitude
66276,2372641,Single Family Residence,Raleigh,27607,5,8,9376,0,2022,2021-03-16,2022-02-23,1850000,324999,344,-1525001,35.8014,-78.6877
113526,2410217,Single Family Residence,Whitsett,27377,5,7,6983,11,1883,2021-09-27,2023-01-09,1300000,3240000,469,1940000,36.0330,-79.5972
26172,2375877,Single Family Residence,Wake Forest,27587,5,5,7200,6,2005,2021-04-05,2021-05-27,1800000,2900000,52,1100000,35.9815,-78.5392
93825,2378933,Single Family Residence,Raleigh,27615,5,7,6289,0,2022,2021-04-20,2022-09-28,2550000,3577591,526,1027591,35.8887,-78.6393
148322,2495996,Single Family Residence,Apex,27502,4,6,7266,1,2023,2023-02-21,2023-11-21,1755900,2598548,273,842648,35.7225,-78.8408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93824,2433596,Single Family Residence,Chapel Hill,27514,2,1,1214,6,1932,2022-02-26,2022-07-08,4300000,3550000,132,-750000,35.9203,-79.0372
158135,2532421,Single Family Residence,Wake Forest,27587,5,6,8767,11,2008,2023-09-15,2024-02-16,4900000,4080467,154,-819533,35.9815,-78.5392
93827,2456529,Single Family Residence,Burlington,27215,6,14,17988,1,1996,2022-06-17,2022-08-29,5900000,4702000,73,-1198000,36.0318,-79.4889
138379,2357404,Single Family Residence,Raleigh,27609,6,11,20283,1,2000,2020-12-10,2023-08-15,4995000,3000000,978,-1995000,35.8480,-78.6317


In [55]:
#Change the 'Latitude' column name to 'Lat'
combined_df = combined_df.rename(columns={"latitude": "Lat"})

#Change the 'Longitude' column name to 'Lon'
combined_df = combined_df.rename(columns={"longitude": "Lon"})

#Change the 'Total Living Area SqFt' column name to 'SqFt'
combined_df = combined_df.rename(columns={"Total Living Area SqFt": "SqFt"})

#Change the 'Profit' column name to 'Over Asking'  
combined_df = combined_df.rename(columns={"Profit": "Over Asking"})

#Change the 'Days Between' column name to 'Days on Market'
combined_df = combined_df.rename(columns={"Days Between": "Days on Market"})


combined_df.head()

,MLS #,Property Type,City,Zip,Bedrooms,Total Baths,SqFt,Acres,Year Built,List Date,Closing Date,List Price,Sold Price,Days on Market,Over Asking,Lat,Lon
66276,2372641,Single Family Residence,Raleigh,27607,5,8,9376,0,2022,2021-03-16,2022-02-23,1850000,324999,344,-1525001,35.8014,-78.6877
113526,2410217,Single Family Residence,Whitsett,27377,5,7,6983,11,1883,2021-09-27,2023-01-09,1300000,3240000,469,1940000,36.0330,-79.5972
26172,2375877,Single Family Residence,Wake Forest,27587,5,5,7200,6,2005,2021-04-05,2021-05-27,1800000,2900000,52,1100000,35.9815,-78.5392
93825,2378933,Single Family Residence,Raleigh,27615,5,7,6289,0,2022,2021-04-20,2022-09-28,2550000,3577591,526,1027591,35.8887,-78.6393
148322,2495996,Single Family Residence,Apex,27502,4,6,7266,1,2023,2023-02-21,2023-11-21,1755900,2598548,273,842648,35.7225,-78.8408


In [56]:
combined_df.isnull().sum()

MLS #               0
Property Type       0
City                0
Zip                 0
Bedrooms            0
Total Baths         0
SqFt                0
Acres               0
Year Built          0
List Date           0
Closing Date        0
List Price          0
Sold Price          0
Days on Market      0
Over Asking         0
Lat               139
Lon               139
dtype: int64

In [57]:
#Drop the rows with missing values
combined_df.dropna(inplace=True)

combined_df.isnull().sum()

MLS #             0
Property Type     0
City              0
Zip               0
Bedrooms          0
Total Baths       0
SqFt              0
Acres             0
Year Built        0
List Date         0
Closing Date      0
List Price        0
Sold Price        0
Days on Market    0
Over Asking       0
Lat               0
Lon               0
dtype: int64

In [58]:
# Count the number of 0 values in the 'Lat' and 'Lon' columns
num_zeros = combined_df['Lon'].value_counts().get(0, 0)

print(num_zeros)

0


In [ ]:
# Copy the data frame to a new data frame
sfr_df = combined_df.copy()

# Keep only the 'Single Family Residence' values in column 'Property Type'
sfr_df = sfr_df[sfr_df['Property Type'] == 'Single Family Residence']

# Drop the 'Property Type' column
sfr_df = sfr_df.drop(columns=['Property Type'])


sfr_df.head()

In [ ]:
# Remove cities with counts less than 10
sfr_df = sfr_df[sfr_df['City'].map(sfr_df['City'].value_counts()) > 1000]
sfr_df['City'].value_counts()

In [ ]:
# Remove unreliable data
sfr_df = sfr_df[sfr_df['Bedrooms'] <= 12]
sfr_df = sfr_df[sfr_df['Bedrooms'] >= 1]
sfr_df = sfr_df[sfr_df['Total Baths'] <= 12]
sfr_df = sfr_df[sfr_df['Total Baths'] >= 1]
sfr_df = sfr_df[sfr_df['SqFt'] >= 100]
sfr_df = sfr_df[sfr_df['Year Built'] >= 1700]
sfr_df = sfr_df[sfr_df['Year Built'] <= 2024]
sfr_df = sfr_df[sfr_df['Sold Price'] >= 50000]

In [ ]:
sfr_df.describe()

In [ ]:
# Extracting year and month from List Date and Closing Date
housing_data['List Year'] = housing_data['List Date'].dt.year
housing_data['List Month'] = housing_data['List Date'].dt.month
housing_data['Closing Year'] = housing_data['Closing Date'].dt.year
housing_data['Closing Month'] = housing_data['Closing Date'].dt.month

housing_data.head()

In [ ]:
# Feature Engineering
housing_data['Price Increase Percentage'] = (housing_data['Sold Price'] - housing_data['List Price']) / housing_data['List Price']
housing_data['Age of Property'] = housing_data['Closing Year'] - housing_data['Year Built']

housing_data.head()

In [ ]:
# Scaling numerical features
scaler = StandardScaler()
numerical_features = ['List Price', 'SqFt', 'Acres', 'Days on Market', 'Price Increase Percentage', 'Age of Property', 'Lat', 'Lon']
housing_data[numerical_features] = scaler.fit_transform(housing_data[numerical_features])

housing_data.head()

In [ ]:
# Create a one-hot encoder
encoder = OneHotEncoder(sparse=False)

# Reshape the "City" column to a 2D array
city_column = housing_data['City'].values.reshape(-1, 1)

# Encode the "City" column
city_encoded = encoder.fit_transform(city_column)

# Create a DataFrame with the encoded values
city_encoded_df = pd.DataFrame(city_encoded, columns=encoder.get_feature_names(['City']))

# Concatenate the encoded DataFrame with the original DataFrame
housing_data_encoded = pd.concat([housing_data, city_encoded_df], axis=1)

housing_data_encoded.head()

In [ ]:
# Target variable analysis
plt.figure(figsize=(8, 6))
sns.histplot(housing_data['Sold Price'], kde=True)
plt.title('Distribution of Sold Price')
plt.show()

In [ ]:
#Create a correlation matrix to see what the correlation is for Price per SqFt
corr_matrix = housing_data.corr()
corr_matrix["Sold Price"].sort_values(ascending=False)

In [ ]:
# Handling outliers
def handle_outliers(df, column):
    '''
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    '''
    #df[np.abs(stats.zscore(df[column])) < 3]
    #housing_data = housing_data[np.abs(stats.zscore(housing_data[col])) < 3]


for col in ['List Price', 'Sold Price', 'SqFt', 'Days on Market', "Bedrooms", "Total Baths", "Age of Property"]:
    #handle_outliers(housing_data, col)
    housing_data = housing_data[np.abs(stats.zscore(housing_data[col])) < 3]

housing_data.head()

In [ ]:
# Round sold price to the nearest $500 increment
housing_data['Sold Price'] = housing_data['Sold Price'].apply(lambda x: round(x/500)*500)

In [ ]:
#Drop the MLS # Column
housing_data.drop('MLS #', axis=1, inplace=True)

#Drop the City Column
housing_data.drop('City', axis=1, inplace=True)

In [59]:
# Save the DataFrame to a CSV file
sfr_df.to_csv('Clean_Data/mls_sfr_cleaned.csv', index=False)